In [1]:
import torch
import torchaudio
import torchaudio.transforms as T
from torch.utils.data import DataLoader, Dataset
from pathlib import Path
import os
from speechbrain.pretrained import EncoderClassifier, HIFIGAN
from tqdm import tqdm

c:\Users\mmc\miniconda3\envs\unlearn\lib\inspect.py:869: UserWarning: Module 'speechbrain.pretrained' was deprecated, redirecting to 'speechbrain.inference'. Please update your script. This is a change from SpeechBrain 1.0. See: https://github.com/speechbrain/speechbrain/releases/tag/v1.0.0
  if ismodule(module) and hasattr(module, '__file__'):
C:\Users\mmc\AppData\Local\Temp\ipykernel_53896\673856426.py:7: UserWarning: Module 'speechbrain.pretrained' was deprecated, redirecting to 'speechbrain.inference'. Please update your script. This is a change from SpeechBrain 1.0. See: https://github.com/speechbrain/speechbrain/releases/tag/v1.0.0
  from speechbrain.pretrained import EncoderClassifier, HIFIGAN


In [2]:
# --- 1. (이전 단계) 데이터로더 설정 ---
# (CustomLibriSpeechDataset, collate_fn_custom_with_lengths 정의 필요)
class CustomLibriSpeechDataset(Dataset):
    """
    사용자의 디렉터리 구조(예: ./data/train/dev-clean)를 
    직접 읽어오는 커스텀 데이터셋 클래스
    """
    def __init__(self, root_dir):
        # LibriSpeech는 .flac 확장자를 사용합니다.
        self.file_paths = sorted(list(Path(root_dir).rglob("*.flac")))
        
        if not self.file_paths:
            raise FileNotFoundError(f"'{root_dir}'에서 .flac 파일을 찾을 수 없습니다. 경로를 확인하세요.")
            
        # 화자 ID(문자열)를 정수(integer)로 매핑합니다.
        self.speaker_map = {}
        self.file_list_with_speaker_id = []
        
        speaker_counter = 0
        
        for file_path in self.file_paths:
            # 파일 경로에서 화자 ID 추출
            # 예: .../dev-clean/84/121123/84-121123.flac
            #       -> 부모의 부모 폴더 이름('84')이 화자 ID
            speaker_id_str = file_path.parent.parent.name
            
            # 화자 ID를 정수로 변환
            if speaker_id_str not in self.speaker_map:
                self.speaker_map[speaker_id_str] = speaker_counter
                speaker_counter += 1
            
            speaker_id_int = self.speaker_map[speaker_id_str]
            
            # (파일 경로, 정수형 화자 ID) 튜플로 저장
            self.file_list_with_speaker_id.append((str(file_path), speaker_id_int))
            
    def __len__(self):
        # 데이터셋의 총 샘플 수
        return len(self.file_list_with_speaker_id)
        
    def __getitem__(self, idx):
        # idx번째 샘플을 반환
        file_path, speaker_id = self.file_list_with_speaker_id[idx]
        
        # 오디오 로드
        waveform, sample_rate = torchaudio.load(file_path)
        
        # (파형, 샘플레이트, 화자ID) 반환
        return waveform, sample_rate, speaker_id

def collate_fn_custom_with_lengths(batch):
    """
    길이가 다른 오디오 샘플들을 묶고, '실제 길이'를 반환하는 함수
    """
    waveforms = []
    speaker_ids = []
    lengths = []  # <-- [추가] 실제 길이를 저장할 리스트
    
    for sample in batch:
        waveform, sample_rate, speaker_id = sample
        
        waveforms.append(waveform.squeeze(0)) # (1, Length) -> (Length)
        speaker_ids.append(speaker_id)
        lengths.append(waveform.shape[1]) # <-- [추가] 패딩 전 실제 길이
        
    # 패딩(Padding) 처리
    waveforms_padded = torch.nn.utils.rnn.pad_sequence(
        waveforms, batch_first=True, padding_value=0.
    )
    
    # 화자 ID와 길이를 텐서로 변환
    speaker_ids_tensor = torch.tensor(speaker_ids, dtype=torch.int64)
    lengths_tensor = torch.tensor(lengths, dtype=torch.int64) # <-- [추가]
    
    # (패딩된 파형, 화자 ID, 실제 길이 텐서) 반환
    return waveforms_padded, speaker_ids_tensor, lengths_tensor

data_root_dev = "./data/train/dev-clean"
batch_size = 1
try:
    librispeech_dataset_dev = CustomLibriSpeechDataset(root_dir=data_root_dev)
    dev_loader = DataLoader(
        dataset=librispeech_dataset_dev,
        batch_size=batch_size,
        shuffle=True,
        collate_fn=collate_fn_custom_with_lengths
    )
    print("DataLoader (길이 반환) 준비 완료.")
except Exception as e:
    print(f"[치명적 오류] 1단계 데이터셋 로드 실패: {e}")
    # (Jupyter Notebook에서는 exit() 대신 SystemExit 예외를 발생)
    raise SystemExit("데이터로더 생성 실패.")


DataLoader (길이 반환) 준비 완료.


In [3]:
# --- 2. 모델 및 변환기 로드 ---
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

# 2-1. 화자 인코더 (ECAPA-TDNN, 16kHz)
try:
    speaker_model = EncoderClassifier.from_hparams(
        source="speechbrain/spkrec-ecapa-voxceleb", 
        run_opts={"device": device}
    )
    speaker_model.eval()
    print("공격 대상 모델 (ECAPA-TDNN) 로드 완료.")
    
    # [수정] 모듈 분리
    feature_extractor = speaker_model.mods.compute_features
    embedding_model = speaker_model.mods.embedding_model
    feature_extractor.eval()
    embedding_model.eval()
    
except Exception as e:
    print(f"[치명적 오류] 2-1단계 화자 인코더 로드 실패: {e}")
    raise SystemExit("speaker_model 로드 실패.")

# 2-2. 뉴럴 보코더 (HiFi-GAN, LibriTTS 16kHz)
try:
    vocoder = HIFIGAN.from_hparams(
        source="speechbrain/tts-hifigan-libritts-16kHz", # <-- [수정됨]
        run_opts={"device": device}
    )
    vocoder.eval()
    print("뉴럴 보코더 (HiFi-GAN LibriTTS-16kHz) 로드 완료.")
except Exception as e:
    print(f"[치명적 오류] 2-2단계 보코더 로드 실패: {e}")
    raise SystemExit("vocoder 로드 실패.")

# --- 2-3. 표준 멜 스펙트로그램 변환기 (수정됨) ---
try:
    from speechbrain.lobes.models.FastSpeech2 import mel_spectogram
    print("SpeechBrain 전용 멜 스펙트로그램 함수 로드 완료.")
except ImportError:
    print("[치명적 오류] 'speechbrain.lobes.models.FastSpeech2'에서 'mel_spectogram'을 찾을 수 없습니다.")
    raise SystemExit("SpeechBrain 멜 함수 로드 실패.")

# 보코더 훈련에 사용된 정확한 파라미터
SAMPLE_RATE = 16000
HOP_LENGTH = 256
WIN_LENGTH = 1024
N_MELS = 80
N_FFT = 1024
F_MIN = 0.0
F_MAX = 8000.0
POWER = 1.0 
COMPRESSION = True

Using device: cuda


c:\Users\mmc\miniconda3\envs\unlearn\lib\site-packages\speechbrain\utils\autocast.py:188: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  wrapped_fwd = torch.cuda.amp.custom_fwd(fwd, cast_inputs=cast_inputs)


공격 대상 모델 (ECAPA-TDNN) 로드 완료.


c:\Users\mmc\miniconda3\envs\unlearn\lib\site-packages\torch\nn\utils\weight_norm.py:143: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)


뉴럴 보코더 (HiFi-GAN LibriTTS-16kHz) 로드 완료.
SpeechBrain 전용 멜 스펙트로그램 함수 로드 완료.


In [4]:
# --- 3. 원본 데이터 및 "기준점" 추출 (수정됨) ---
print("\n3단계: 데이터로더에서 배치 가져오는 중...")
try:
    # (dev_loader가 이미 정의되어 있으므로 next()만 다시 호출)
    waveforms, speaker_ids, lengths = next(iter(dev_loader))
    waveforms, speaker_ids, lengths = waveforms.to(device), speaker_ids.to(device), lengths.to(device)
    
    # 1. 원본 "임베딩" (공격의 목표점)
    with torch.no_grad():
        rel_lengths = lengths.float() / waveforms.shape[1]
        original_embeddings = speaker_model.encode_batch(waveforms, wav_lens=rel_lengths).squeeze(1)

    # 2. 원본 "멜 스펙트로그램" (공격의 시작점)
    with torch.no_grad():
        wav_input = waveforms.squeeze(0) # [B, T_wav] -> [T_wav]
        
        # SpeechBrain 함수 호출 (출력: [Mels, T] 즉 [80, 306])
        original_mels, _ = mel_spectogram(
            audio=wav_input,
            sample_rate=SAMPLE_RATE,
            hop_length=HOP_LENGTH,
            win_length=WIN_LENGTH,
            n_mels=N_MELS,
            n_fft=N_FFT,
            f_min=F_MIN,
            f_max=F_MAX,
            power=POWER,
            normalized=False,
            min_max_energy_norm=True,
            norm="slaney",
            mel_scale="slaney",
            compression=COMPRESSION
        )
        
        # [!!! --- 핵심 수정 --- !!!]
        # .transpose(0, 1)를 제거하고 .unsqueeze(0)만 사용하여 배치 차원 추가
        # [Mels, T] -> [B, Mels, T]
        original_mels = original_mels.unsqueeze(0) 
        # (이전 코드: original_mels = original_mels.transpose(0, 1).unsqueeze(0))
        
    print(f"배치 로드 완료. 파형 Shape: {waveforms.shape}")
    print(f"원본 임베딩(타겟) Shape: {original_embeddings.shape}")
    print(f"원본 멜(공격대상) Shape: {original_mels.shape}") # [1, 80, 306]이 출력되어야 함

except StopIteration:
    print("[치명적 오류] 3단계 오류: 데이터로더가 비어있습니다. (StopIteration)")
    raise SystemExit("데이터 로드 실패.")
except Exception as e:
    print(f"[치명적 오류] 3단계 데이터 처리 중 오류 발생: {e}")
    raise SystemExit("데이터 처리 실패.")
    



3단계: 데이터로더에서 배치 가져오는 중...
배치 로드 완료. 파형 Shape: torch.Size([1, 77920])
원본 임베딩(타겟) Shape: torch.Size([1, 192])
원본 멜(공격대상) Shape: torch.Size([1, 80, 305])


In [5]:
# --- 3.5. [필수] 베이스라인 검증 ---
print("\n3.5단계: 베이스라인 검증 시작...")
output_dir = "attack_results"
os.makedirs(output_dir, exist_ok=True)
try:
    with torch.no_grad():
        # [수정] 이제 original_mels는 올바른 [1, 80, 306] Shape을 가짐
        baseline_waveform = vocoder.mods.generator(original_mels).squeeze(1).cpu()

    # (이하 파일 저장 코드 동일)
    torchaudio.save(
        os.path.join(output_dir, "ref/0_original_raw.wav"), 
        waveforms[0].cpu().unsqueeze(0), 
        SAMPLE_RATE
    )
    torchaudio.save(
        os.path.join(output_dir, "output/0_baseline_vocoder.wav"), 
        baseline_waveform[0].unsqueeze(0), 
        SAMPLE_RATE
    )
    print("베이스라인 검증 완료. '0_baseline_vocoder.wav'를 다시 확인하세요.")
    print(">>> '0_original_raw.wav'와 소리가 거의 동일해야 합니다.")
    
except Exception as e:
    print(f"[치명적 오류] 3.5단계 베이스라인 검증 중 오류: {e}")
    raise SystemExit("베이스라인 검증 실패.")

print("\n--- 2-3/3/3.5단계 (준비) 완료. 4단계(PGD 공격)를 진행할 수 있습니다. ---")


3.5단계: 베이스라인 검증 시작...
베이스라인 검증 완료. '0_baseline_vocoder.wav'를 다시 확인하세요.
>>> '0_original_raw.wav'와 소리가 거의 동일해야 합니다.

--- 2-3/3/3.5단계 (준비) 완료. 4단계(PGD 공격)를 진행할 수 있습니다. ---


In [6]:
# --- 4. PGD 공격 구현 (수정된 파이프라인) ---
print("\n--- 4단계: PGD 공격 시작 (Target: Standard Mel) ---")

# PGD 하이퍼파라미터 (로그-멜 스펙트로그램 기준이므로 epsilon을 작게 시작)
epsilon = 0.05   # [조정 필요] 최대 노이즈 크기
learning_rate = 0.005 # [조정 필요] 스텝 크기
iterations = 40    # 반복 횟수

# 1. 노이즈 텐서(delta) 랜덤 초기화 (original_mels와 동일한 크기)
#    (epsilon 범위 내의 랜덤 값으로 시작)
delta_init = (torch.rand_like(original_mels) * 2 - 1) * epsilon
delta = delta_init.to(device)
delta.requires_grad = True # 이 텐서의 그래디언트를 계산

# 2. 손실 함수 (MSE Loss)
mse_loss = torch.nn.MSELoss()
pbar = tqdm(range(iterations), desc="PGD Attack Loop")

for i in pbar: 
    try:
        # 3. 계산 그래프 생성 강제
        with torch.enable_grad(): 
            # 4. 적대적 멜 스펙트로그램 생성
            adv_mels = original_mels + delta
            
            # [수정된 파이프라인]
            # 5. 멜 -> 파형 (Differentiable Vocoder)
            #    (입력 Shape: [B, N_Mels, T], HiFi-GAN G 입력과 일치)
            adv_waveform = vocoder.mods.generator(adv_mels).squeeze(1) 
            
            # 6. 생성된 파형의 '새로운 상대 길이' 계산
            #    (현재 batch_size=1이므로 길이는 항상 1.0)
            if adv_waveform.shape[0] == 0 or adv_waveform.shape[1] == 0:
                tqdm.write("!!오류: 보코더가 빈 오디오를 생성했습니다.")
                continue
            
            # (ECAPA-TDNN의 encode_batch는 wav_lens를, 
            #  내부 embedding_model은 상대 길이를 받음)
            adv_rel_lengths = torch.tensor([1.0] * adv_waveform.shape[0], device=device)

            # 7. 파형 -> 화자 인코더 피처 (Feature Extractor)
            adv_features_re_extracted = feature_extractor(adv_waveform)
            
            # 8. 피처 -> 임베딩 (Embedding Model)
            adv_embeddings = embedding_model(adv_features_re_extracted, adv_rel_lengths).squeeze(1)
            
            # 9. 손실 계산 (원본 임베딩과의 거리를 최대화)
            loss = -mse_loss(adv_embeddings, original_embeddings.detach())
        
        # 10. 그래디언트 계산
        speaker_model.zero_grad()
        vocoder.zero_grad() 
        loss.backward() 

        # 11. PGD 업데이트 (Loss가 음수이므로 -)
        delta.data = delta.data - learning_rate * delta.grad.sign()
        
        # 12. 프로젝션 (L-infinity): delta가 epsilon 예산을 넘지 않도록 제한
        delta.data = torch.clamp(delta.data, -epsilon, epsilon)
        
        # 13. delta 그래디언트 초기화
        delta.grad.zero_()

        pbar.set_description(f"PGD Attack Loop [Loss: {loss.item():.8f}]")

    except RuntimeError as e:
        tqdm.write(f"\n[오류 발생] Iteration {i+1}: PGD 루프 실패.")
        tqdm.write(f"오류 메시지: {e}")
        if "out of memory" in str(e).lower():
            tqdm.write(">>> [제안] CUDA OutOfMemory. batch_size=1인지 확인하세요.")
        break 

pbar.close()
print("--- PGD 공격 완료 ---")


--- 4단계: PGD 공격 시작 (Target: Standard Mel) ---


PGD Attack Loop [Loss: -760.39324951]: 100%|██████████| 40/40 [00:01<00:00, 27.58it/s]

--- PGD 공격 완료 ---


In [7]:
# --- 5. 공격 성공 여부 검증 ---
print("\n--- 5단계: 공격 검증 시작 ---")
try:
    with torch.no_grad():
        # 1. 최종 적대적 멜 생성
        final_adv_mels = (original_mels + delta).detach()
        
        # 2. 최종 적대적 '오디오 파형' 생성 (비인지성 검증 대상!)
        final_adv_waveform = vocoder.mods.generator(final_adv_mels).squeeze(1).cpu()

        # 3. 최종 적대적 '임베딩' 계산 (공격 성공률 검증)
        #    (PGD 루프와 동일한 과정을 no_grad로 수행)
        adv_wav_gpu = final_adv_waveform.to(device)
        adv_rel_lengths = torch.tensor([1.0] * adv_wav_gpu.shape[0], device=device)
        adv_features_re = feature_extractor(adv_wav_gpu)
        final_adv_embeddings = embedding_model(adv_features_re, adv_rel_lengths).squeeze(1).cpu()

    # 코사인 유사도 검증
    cos_sim = torch.nn.functional.cosine_similarity(
        original_embeddings.cpu(), 
        final_adv_embeddings, 
        dim=1
    )

    print("\n--- 공격 검증 결과 (Domain-Aligned) ---")
    print(f"원본 vs 적대적 임베딩 간 코사인 유사도 (1.0 = 동일, 낮을수록 성공):")
    print(f"  -> {cos_sim.item():.4f}") # (batch_size=1이므로 .mean() 대신 .item())

    # 베이스라인 비교 (다른 화자와의 유사도)
    shuffled_orig_embeddings = original_embeddings[torch.randperm(original_embeddings.size(0))]
    baseline_cos_sim = torch.nn.functional.cosine_similarity(
        original_embeddings.cpu(), 
        shuffled_orig_embeddings.cpu(),
        dim=1
    )
    print(f"참고: 원본 vs 다른 화자 간 평균 유사도 (베이스라인): {baseline_cos_sim.mean().item():.4f}")


    # 오디오 파일 저장 (비교 청취용)
    torchaudio.save(
        os.path.join(output_dir, "attack/1_adversarial_attack.wav"), 
        final_adv_waveform[0].unsqueeze(0), 
        SAMPLE_RATE
    )
    print(f"\n결과 오디오 파일 '1_adversarial_attack.wav'가 '{output_dir}' 폴더에 저장되었습니다.")
    print("\n>>> [최종 검증] '0_baseline_vocoder.wav'와 '1_adversarial_attack.wav'를 비교 청취하세요.")
    print(">>> 1. 두 파일의 음질이 유사합니까? (비인지성 성공)")
    print(">>> 2. 코사인 유사도 값이 (e.g., 0.9) 베이스라인(e.g., 0.2)에 가깝게 떨어졌습니까? (공격 성공)")

except Exception as e:
    print(f"[치명적 오류] 5단계 검증 중 오류: {e}")


--- 5단계: 공격 검증 시작 ---

--- 공격 검증 결과 (Domain-Aligned) ---
원본 vs 적대적 임베딩 간 코사인 유사도 (1.0 = 동일, 낮을수록 성공):
  -> 0.5193
참고: 원본 vs 다른 화자 간 평균 유사도 (베이스라인): 1.0000

결과 오디오 파일 '1_adversarial_attack.wav'가 'attack_results' 폴더에 저장되었습니다.

>>> [최종 검증] '0_baseline_vocoder.wav'와 '1_adversarial_attack.wav'를 비교 청취하세요.
>>> 1. 두 파일의 음질이 유사합니까? (비인지성 성공)
>>> 2. 코사인 유사도 값이 (e.g., 0.9) 베이스라인(e.g., 0.2)에 가깝게 떨어졌습니까? (공격 성공)


In [8]:
import torch
import numpy as np
from tqdm import tqdm
import os
import torchaudio

print("\n--- 5단계: 일반화 검증 시작 ---")

# --- 테스트 파라미터 ---
max_samples_to_test = 40  # [조정 가능] 검증할 총 샘플 수 (전체는 수천 개)
pgd_iterations = 40       # PGD 반복 횟수
pgd_epsilon = 0.05        # 멜 노이즈 크기 (이전과 동일)
pgd_learning_rate = 0.005 # PGD 스텝 크기 (이전과 동일)

# --- PGD 공격에 필요한 파라미터 (이전 단계에서 복사) ---
SAMPLE_RATE = 16000
HOP_LENGTH = 256
# (mel_spectogram 함수가 이미 정의되어 있다고 가정)

# --- 결과 저장용 리스트 ---
all_cosine_similarities = []

# --- 데이터로더 순회 ---
# (이전 단계에서 정의된 dev_loader 사용)
print(f"총 {max_samples_to_test}개의 샘플에 대해 일반화 검증을 시작합니다...")
pbar_loader = tqdm(dev_loader, total=max_samples_to_test, desc="Generalization Test")

for i, batch in enumerate(pbar_loader):
    if i >= max_samples_to_test:
        tqdm.write(f"{max_samples_to_test}개 샘플 테스트 완료. 루프를 중단합니다.")
        break  # 테스트 샘플 수에 도달하면 중지

    try:
        # --- 3단계 (적응): 원본 데이터 및 기준점 추출 ---
        pbar_loader.set_description(f"Sample {i+1} [Step 3: Loading data]")
        
        waveforms, speaker_ids, lengths = batch
        waveforms, speaker_ids, lengths = waveforms.to(device), speaker_ids.to(device), lengths.to(device)

        with torch.no_grad():
            # 1. 원본 임베딩 (공격 타겟)
            rel_lengths = lengths.float() / waveforms.shape[1]
            original_embeddings = speaker_model.encode_batch(waveforms, wav_lens=rel_lengths).squeeze(1)

            # 2. 원본 멜 스펙트로그램 (공격 대상)
            wav_input = waveforms.squeeze(0)
            original_mels, _ = mel_spectogram(
                audio=wav_input, sample_rate=SAMPLE_RATE, hop_length=HOP_LENGTH,
                win_length=1024, n_mels=80, n_fft=1024, f_min=0.0, f_max=8000.0,
                power=1.0, normalized=False, min_max_energy_norm=True,
                norm="slaney", mel_scale="slaney", compression=True
            )
            original_mels = original_mels.unsqueeze(0) # [1, Mels, T]

        # --- 4단계 (적응): PGD 공격 루프 ---
        pbar_loader.set_description(f"Sample {i+1} [Step 4: PGD Attack]")

        delta_init = (torch.rand_like(original_mels) * 2 - 1) * pgd_epsilon
        delta = delta_init.to(device)
        delta.requires_grad = True
        mse_loss = torch.nn.MSELoss()

        # (내부 PGD 루프는 tqdm을 표시하지 않음)
        for _ in range(pgd_iterations):
            with torch.enable_grad():
                adv_mels = original_mels + delta
                adv_waveform = vocoder.mods.generator(adv_mels).squeeze(1)
                
                if adv_waveform.shape[0] == 0 or adv_waveform.shape[1] == 0:
                    continue # 빈 오디오 생성 시 스킵
                
                adv_rel_lengths = torch.tensor([1.0] * adv_waveform.shape[0], device=device)
                adv_features_re = feature_extractor(adv_waveform)
                adv_embeddings = embedding_model(adv_features_re, adv_rel_lengths).squeeze(1)
                loss = -mse_loss(adv_embeddings, original_embeddings.detach())
            
            speaker_model.zero_grad()
            vocoder.zero_grad()
            loss.backward()
            
            delta.data = delta.data - pgd_learning_rate * delta.grad.sign()
            delta.data = torch.clamp(delta.data, -pgd_epsilon, pgd_epsilon)
            delta.grad.zero_()

        # --- 5단계 (적응): 최종 검증 ---
        pbar_loader.set_description(f"Sample {i+1} [Step 5: Verifying]")
        
        with torch.no_grad():
            final_adv_mels = (original_mels + delta).detach()
            final_adv_waveform = vocoder.mods.generator(final_adv_mels).squeeze(1).cpu()
            
            adv_wav_gpu = final_adv_waveform.to(device)
            adv_rel_lengths_final = torch.tensor([1.0] * adv_wav_gpu.shape[0], device=device)
            adv_features_re_final = feature_extractor(adv_wav_gpu)
            final_adv_embeddings = embedding_model(adv_features_re_final, adv_rel_lengths_final).squeeze(1).cpu()

            cos_sim = torch.nn.functional.cosine_similarity(
                original_embeddings.cpu(), 
                final_adv_embeddings, 
                dim=1
            )
            
            all_cosine_similarities.append(cos_sim.item())

        # (선택 사항) 매 N번째 샘플마다 공격 오디오 저장
        if i < 5: # 처음 5개 샘플만 저장
            torchaudio.save(
                os.path.join(output_dir, f"{i}_adversarial_attack.wav"), 
                final_adv_waveform[0].unsqueeze(0), 
                SAMPLE_RATE
            )
            
        # Tqdm 진행률 표시줄 업데이트
        pbar_loader.set_description(f"Generalization Test [Avg CosSim: {np.mean(all_cosine_similarities):.4f}]")

    except RuntimeError as e:
        tqdm.write(f"\n[오류 발생] Sample {i+1} 처리 중 오류: {e}")
        if "out of memory" in str(e).lower():
            tqdm.write(">>> CUDA OutOfMemory. 오디오가 너무 길 수 있습니다. 다음 샘플로 넘어갑니다.")
            continue # OOM 발생 시 다음 샘플로
        else:
            break # 다른 오류 시 중단
    except Exception as e:
        tqdm.write(f"\n[알 수 없는 오류] Sample {i+1} 처리 중 오류: {e}")
        continue

pbar_loader.close()

# --- 최종 결과 리포트 ---
print("\n--- 일반화 검증 완료 ---")
if all_cosine_similarities:
    avg_sim = np.mean(all_cosine_similarities)
    min_sim = np.min(all_cosine_similarities)
    max_sim = np.max(all_cosine_similarities)
    
    print(f"테스트한 총 샘플 수: {len(all_cosine_similarities)}")
    print(f"평균 코사인 유사도: {avg_sim:.4f}")
    print(f"최저 유사도 (가장 공격 성공): {min_sim:.4f}")
    print(f"최고 유사도 (가장 공격 실패): {max_sim:.4f}")
else:
    print("검증이 완료된 샘플이 없습니다. 오류 로그를 확인하세요.")


--- 5단계: 일반화 검증 시작 ---
총 40개의 샘플에 대해 일반화 검증을 시작합니다...


Generalization Test [Avg CosSim: 0.4619]: 100%|██████████| 40/40 [01:05<00:00,  1.63s/it]

40개 샘플 테스트 완료. 루프를 중단합니다.

--- 일반화 검증 완료 ---
테스트한 총 샘플 수: 40
평균 코사인 유사도: 0.4619
최저 유사도 (가장 공격 성공): 0.0992
최고 유사도 (가장 공격 실패): 0.6212
